In [23]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt



secret_key = '0rDQSrIkvK4cEMqHtq24XCQ2bfS1fexhlqBHcNyJ'
api_key = 'PK4TO061KMGMCH7LNQOB'

In [24]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)

# tickers = df['Symbol']

# apple = df[df['Symbol'] == 'AAPL']

In [43]:
# PRICE CHANGE AND ROLLING CHANGE SCREENING

def get_price_change_and_rolling_avg(ticker: str):
    """
    Fetches the price change from today to yesterday and the rolling average
    of the price change over the past ~2 weeks for a given ticker.
    
    params:
        ticker (str): The stock ticker symbol.
    
    returns:
        a tuple containing the price change from today to yesterday and
               the 2-week rolling average of price changes.
    """
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=24)
    
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    
    data = data.sort_index()
    
    data['Price_Change'] = (data['Close'].diff() / data['Close'].shift(1)) * 100
    today_price_change = data['Price_Change'].iloc[-1]

    rolling_avg = data['Price_Change'].rolling(window=min(14, len(data))).mean().iloc[-1]
    
    return today_price_change, rolling_avg


def fetch_option_chain(api_key: str, secret_key: str, underlying_symbol: str):
    """
    Fetches the option chain for a given underlying symbol using Alpaca's API.
    """
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{underlying_symbol}?feed=indicative&limit=100&expiration_date=2025-02-07"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        snapshots = data.get('snapshots', {})
        return snapshots
    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None

def parse_option_chain(option_chain):
    """
    Parses the option chain data and returns a DataFrame with the required columns.
    """
    parsed_data = []
    
    for symbol, details in option_chain.items():
        expiration_start = len(symbol) - 15
        ticker = symbol[:expiration_start]
        expiration_date = f"20{symbol[expiration_start:expiration_start+2]}-{symbol[expiration_start+2:expiration_start+4]}-{symbol[expiration_start+4:expiration_start+6]}"
        option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
        strike_price = int(symbol[expiration_start+7:]) / 1000  
        
        greeks = details.get("greeks", {}) or {}
        implied_volatility = details.get("impliedVolatility", None)
        latest_quote = details.get("latestQuote", {})
        
        parsed_data.append({
            "ticker": ticker,
            "expiration_date": expiration_date,
            "option_type": option_type,
            "strike_price": strike_price,
            "delta": greeks.get("delta"),
            "gamma": greeks.get("gamma"),
            "rho": greeks.get("rho"),
            "theta": greeks.get("theta"),
            "vega": greeks.get("vega"),
            "implied_volatility": implied_volatility,
            "ask_price": latest_quote.get("ap"),
            "ask_size": latest_quote.get("as"),
            "bid_price": latest_quote.get("bp"),
            "bid_size": latest_quote.get("bs")
        })
    
    return pd.DataFrame(parsed_data)

def get_option_chain(api_key: str, secret_key: str, underlying_symbol: str):
    """
    Fetches and parses the option chain data, returning it as a DataFrame.
    """
    option_chain = fetch_option_chain(api_key, secret_key, underlying_symbol)
    if option_chain:
        return parse_option_chain(option_chain)
    else:
        return pd.DataFrame()
    

def gbm(s0=100, mu=0.2, sigma=0.68, deltaT=52, dt=0.1):
    """
    Models a stock price S(t) using the Weiner process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        s: A NumPy array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)
    time_vector = np.linspace(0, deltaT, num=n_step)
    
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    
    weiner_process = sigma * np.random.normal(size=n_step)
    
    s = s0 * np.exp(stock_var + weiner_process)
    
    return s









In [26]:
# SCREENS STOCKS WITH 1-DAY CHANGE OF -2% AND ROLLING AVG OF CHANGE OVER 2 WEEKS OF > 0%
filtered_stocks = []
for index, stock in NASDAQ.iterrows():
    today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
    if (today_change <= -2.00) and (rolling_avg > 0.00): 
        filtered_stocks.append(stock['Symbol'])
        print(f"Ticker: {stock['Symbol']}")
        print(f"Today's price change: {today_change}")
        print(f"2-week rolling average of price changes: {rolling_avg}")



Ticker: WDAY
Today's price change: -2.0082843612816785
2-week rolling average of price changes: 0.15987179595244022
Ticker: DDOG
Today's price change: -2.1342304311686418
2-week rolling average of price changes: 0.06723050081733792
Ticker: MDB
Today's price change: -2.6084071419885952
2-week rolling average of price changes: 0.6457693980138542


In [48]:
option_chain = get_option_chain(api_key, secret_key, 'AAPL')

put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)


# run through gmb simulation to decide if profitable or not
for index, contract in put_chain.iterrows():
    contract['strike_price']

put_chain


,ticker,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size
40,AAPL,2025-02-07,Put,187.5,-0.0075,0.0008,-0.0004,-0.0404,0.0070,0.8118,0.08,4,0.06,41
70,AAPL,2025-02-07,Put,190.0,-0.0111,0.0010,-0.0005,-0.0578,0.0099,0.8226,0.14,56,0.08,1
8,AAPL,2025-02-07,Put,192.5,-0.0139,0.0013,-0.0007,-0.0695,0.0121,0.8121,0.15,190,0.13,9
53,AAPL,2025-02-07,Put,195.0,-0.0138,0.0014,-0.0007,-0.0648,0.0119,0.7658,0.16,648,0.10,11
30,AAPL,2025-02-07,Put,197.5,-0.0173,0.0017,-0.0008,-0.0776,0.0145,0.7543,0.17,1,0.16,121
18,AAPL,2025-02-07,Put,200.0,-0.0165,0.0017,-0.0008,-0.0692,0.0139,0.7023,0.15,1,0.14,22
63,AAPL,2025-02-07,Put,202.5,-0.0207,0.0021,-0.0010,-0.0829,0.0169,0.6904,0.20,200,0.17,98
51,AAPL,2025-02-07,Put,205.0,-0.0272,0.0027,-0.0013,-0.1034,0.0213,0.6855,0.29,5,0.21,1
60,AAPL,2025-02-07,Put,207.5,-0.0316,0.0032,-0.0015,-0.1126,0.0241,0.6611,0.32,99,0.25,121
43,AAPL,2025-02-07,Put,210.0,-0.0382,0.0038,-0.0019,-0.1281,0.0281,0.6435,0.36,3,0.33,1
